# Webinar Exercise: Advanced NLP with ChatGPT API

## Objective
Leverage the ChatGPT API to extract structured insights from unstructured support ticket data.

**Goals:**
- Apply advanced natural language processing (NLP) with ChatGPT.
- Extract metadata such as issue category, urgency, and action items.
- Compare traditional keyword-based methods with AI-driven extraction.


In [1]:
# Load the support ticket dataset
# Update the path with your path
import pandas as pd
df = pd.read_csv('/content/sample_data/support_tickets.csv')
df.head()

,ticket_id,customer_comment,product,channel
0,1001,The battery life is terrible and the device ov...,Smartphone,Email
1,1002,I ordered the product two weeks ago and still ...,Tablet,Web Form
2,1003,The price dropped right after I bought it. Ver...,Headphones,Chat
3,1004,"Customer service was helpful, but the setup in...",Smartwatch,Phone
4,1005,Love the product! The sound quality is amazing...,Speaker,Email


## Step 1: Traditional Keyword-Based Tagging (Baseline)
Try extracting categories (e.g., billing, technical, access) using keywords.

Purpose: Establish a baseline using a simple rule-based classifier. This allows learners to understand limitations of hardcoded heuristics compared to intelligent, context-aware systems like ChatGPT.


In [2]:
# Simple keyword-based rule system for categorizing tickets
def categorize_ticket(text):
    text = text.lower()
    if 'bill' in text or 'charge' in text:
        return 'Billing'
    elif 'error' in text or 'bug' in text:
        return 'Technical'
    elif 'login' in text or 'access' in text:
        return 'Access'
    else:
        return 'Other'

df['keyword_category'] = df['customer_comment'].apply(categorize_ticket)
df[['customer_comment', 'keyword_category']].head()

,customer_comment,keyword_category
0,The battery life is terrible and the device ov...,Other
1,I ordered the product two weeks ago and still ...,Other
2,The price dropped right after I bought it. Ver...,Other
3,"Customer service was helpful, but the setup in...",Other
4,Love the product! The sound quality is amazing...,Other


## Step 2: ChatGPT-Powered Classification
Use ChatGPT to:
- Classify the issue type
- Extract urgency
- Summarize the required action

**Important:** You must have an OpenAI API key set in your environment as `OPENAI_API_KEY`.

Purpose: Replace manual tagging with automated, intelligent categorization using ChatGPT. This approach accounts for context, tone, and indirect language, offering much richer metadata.

In [3]:
# Setup for ChatGPT API
import openai
import os
os.environ["OPENAI_API_KEY"]= "insert your key here"

openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
# Function to query ChatGPT API with a custom prompt
def classify_with_chatgpt(text):
    prompt = f"""
    Analyze the following support ticket and return a JSON with:
    - category (e.g., Billing, Technical, Access, Other)
    - urgency (Low, Medium, High)
    - action_summary (a 1-line summary of recommended action)

    Ticket: \"{text}\"
    """

    client = openai.OpenAI()

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "You are a customer support analyst."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=150
    )
    return response.choices[0].message.content

### Try It: Run ChatGPT on a Sample Ticket

In [5]:
# Example run on a single ticket
sample_ticket = df['customer_comment'][0]
response = classify_with_chatgpt(sample_ticket)
print(response)

```json
{
  "category": "Technical",
  "urgency": "High",
  "action_summary": "Investigate and resolve device overheating and poor battery performance issues."
}
```



Sentiment and Escalation Detection

Extend your ChatGPT-powered classification function to also:
- Detect the customer's **sentiment** (positive, negative, neutral)
- Determine whether the support ticket should be **escalated** based on urgency or tone

You may revise the function to request this additional information from the API.


In [6]:
# Extended function: ChatGPT classification + sentiment + escalation flag

def classify_ticket_with_chatgpt_extended(ticket_text):
    client = openai.OpenAI() # Initialize the client inside the function
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "You are a customer support NLP assistant. Given a support ticket, determine: (1) the issue type, (2) urgency level, (3) customer sentiment (positive, negative, neutral), and (4) whether the issue should be escalated. Return the results in JSON format."},
            {"role": "user", "content": f"Ticket: {ticket_text}"}
        ],
        temperature=0.3,
        max_tokens=300
    )
    return response.choices[0].message.content.strip()

# Try it out
print(classify_ticket_with_chatgpt_extended("I’ve emailed twice about the wrong item being delivered. Still no response. Very frustrating!"))

{
  "issue_type": "Incorrect Item Delivered",
  "urgency_level": "High",
  "customer_sentiment": "Negative",
  "escalation_required": true
}


In [7]:
import json
from pprint import pprint

In [8]:
# Load sample customer feedback
data = [
    "The delivery was late and I couldn't get an update.",
    "Customer support was very helpful and friendly.",
    "My product arrived damaged. Very disappointed.",
    "Great quality and fast service, I’ll order again!",
    "Terrible website experience. Couldn’t complete my purchase."
]
df = pd.DataFrame({'feedback': data})
df

,feedback
0,The delivery was late and I couldn't get an up...
1,Customer support was very helpful and friendly.
2,My product arrived damaged. Very disappointed.
3,"Great quality and fast service, I’ll order again!"
4,Terrible website experience. Couldn’t complete...


In [9]:
# Define the ChatGPT function to extract actionable recommendations
def ask_chatgpt_actionable(feedback):
    prompt = (
        "Based on this customer feedback, what are three specific actions the company "
        "could take to improve customer experience? Please explain briefly:\n" + feedback
    )

    response = openai.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "You are a business analyst helping a company interpret feedback and provide improvement suggestions."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

In [10]:
# Combine all feedback into one string and call the function
all_feedback = "\n".join(df['feedback'].tolist())

# Uncomment this to run live
recommendations = ask_chatgpt_actionable(all_feedback)

# Simulated (mocked) response from API
#with open("/content/sample_data/chatgpt_actionable_recommendations.txt", "r") as f:
#    recommendations = f.read()

print("\033[1mActionable Recommendations:\033[0m\n")
print(recommendations)

Actionable Recommendations:

To improve customer experience based on the feedback provided, the company could consider implementing the following specific actions:

1. **Enhance Delivery and Tracking System**: The feedback about late delivery and lack of updates suggests a need for a more reliable and transparent delivery tracking system. The company could invest in technology that provides real-time tracking updates directly to customers via email or a mobile app. This would not only improve customer satisfaction by keeping them informed but also reduce the workload on customer support to handle tracking inquiries.

2. **Quality Control Measures**: Given that a product was received damaged, it's crucial to implement stricter quality control checks before dispatch. This could involve more rigorous inspections at various stages of the supply chain and possibly reevaluating packaging methods to ensure that products are secured better during transit. This action would likely reduce the in